In [ ]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.6.0


In [5]:
# Dataset is available in Keras
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

*All are gray scale images*

Downscaling the pixels so that each pixel value is from 0 to 1 only


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [ ]:
train_images[0] # Pixel values of first image

In [12]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

Keras

```
1. First Convolution Layer
2. Second Convolution Layer
3. Flatten
4. Dense Layer
5. Adam Optimizer
```



In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8803333044052124

Best val_accuracy So Far: 0.9143333435058594
Total elapsed time: 00h 05m 01s
INFO:tensorflow:Oracle triggered exit


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        179264    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1982528   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 2,163,562
Trainable params: 2,163,562
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1339 - accuracy: 0.9496 - val_loss: 0.2629 - val_accuracy: 0.9172
Epoch 5/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0967 - accuracy: 0.9643 - val_loss: 0.2835 - val_accuracy: 0.9133
Epoch 6/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0697 - accuracy: 0.9739 - val_loss: 0.3008 - val_accuracy: 0.9148
Epoch 7/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0509 - accuracy: 0.9812 - val_loss: 0.3793 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0390 - accuracy: 0.9859 - val_loss: 0.4145 - val_accuracy: 0.9153
Epoch 9/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0301 - accuracy: 0.9892 - val_loss: 0.4525 - val_accuracy: 0.9142
Epoch 10/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0279 - accuracy: 0.9901 - val_loss: 0.4485 

*Accuracy of the model is 99%*